In [ ]:
%pip install plotly

In [134]:
# Constants
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt  
import plotly.graph_objects as go

In [136]:
# Code for Dataset loading and print dataset statistics
#-----write your code below this line---------

movie_data_set= pd.read_csv('C:\\Users\\madha\\OneDrive\\Desktop\\bits\semII\\DRL\\assignment\\TrendMovie.csv')
movie_data_set

df = pd.DataFrame(movie_data_set)

# Add a new column 'Category' based on the 'Age' column
#df['rewards'] = df['rating'].apply(lambda x: 1 if x >= 4 else 0)
df['reward'] = (df['rating'] >= 4).astype(int) 
df = df.dropna(subset=['userId', 'movieId', 'rating'])

df

,userId,movieId,rating,timestamp,reward
0,1,1,4.0,964982703,1
1,1,3,4.0,964981247,1
2,1,6,4.0,964982224,1
3,1,47,5.0,964983815,1
4,1,50,5.0,964982931,1
...,...,...,...,...,...
100831,610,166534,4.0,1493848402,1
100832,610,168248,5.0,1493850091,1
100833,610,168250,5.0,1494273047,1
100834,610,168252,5.0,1493846352,1


In [163]:
class MovieEnvironment():
    # Constructor to initialize environment
    def __init__(self, df):
        self.df = df
        self.n_movies = df['movieId'].nunique()  # Number of unique movies (arms)
        self.users = df['userId'].unique()  # List of unique users
        self.user_id=np.random.choice(self.users)
        print(f"Recommendation for the user :{self.user_id}")
        self.epsilon_1 = 0.1
        self.epsilon_2 = 0.2
        self.epsilon_3 = 0.5
        
        
    
    def step(self, movieId,rating):              
        reward = 1 if rating >= 4 else 0
        self.user_movie_rewards[movieId].append(reward)
        return self.state, reward

    def reset(self):
       self.user_movie_rewards = {}       
       return self.user_movie_rewards


    def compute_reward(self, rating):        
        reward = 1 if rating >= 4 else 0
        return reward
       
    def random_policy(self,iterations=1000):                              
				   # Extract movie IDs for the given user

       total_rewards = []  # Track total rewards for each user
       movie_rewards={}           
       user_movies = df[df['userId'] == self.user_id]
    
       user_movie_ids = user_movies['movieId'].unique()
       
       movie_rewards = {movie: 0 for movie in user_movie_ids}  # Track rewards for each movie		
				   # Run the simulation for the specified number of iterations
       for i in range(iterations):
        # Randomly select a movie (exploration)
        movie_id = np.random.choice(user_movie_ids) # Random movie
        # Get the rating for the user and selected movie
        movie_ratings = user_movies[user_movies['movieId'] == movie_id]
        if not movie_ratings.empty:
         rating = movie_ratings.iloc[0]['rating']
         reward = self.compute_reward(rating)
         # Update rewards and counts
         user_movies.loc[user_movies['movieId'] == movie_id, 'reward'] += reward
         total_rewards.append(reward)
       return np.cumsum(total_rewards)
    def greedy_policy(self,iterations): 
        
        user_movies = df[df['userId'] == self.user_id]     
        user_movie_ids = user_movies['movieId'].unique()

        user_with_rewards=df[(df['userId'] == self.user_id) & (df['rating'] >=4)]
        # Get total count of rows with rating >= 4
        total_count = len(user_with_rewards)
        total_rewards=[]
        movie_with_rewards={movie: 0 for movie in user_movie_ids}
        explore =0
        #print(user_with_rewards.value_counts())
        for movieId in user_movie_ids:
         movie_ratings = user_movies[user_movies['movieId'] == movieId]
         if not movie_ratings.empty:
          rating = movie_ratings.iloc[0]['rating']
          reward = self.compute_reward(rating)
          # Update rewards and counts          
          user_movies.loc[user_movies['movieId'] == movieId, 'reward'] += reward 
         explore= explore+1
         if explore >total_count:
          break 
        max_reward_movie = max(movie_with_rewards.values())
        max_rewad_filter=[movie_id for movie_id, reward in movie_with_rewards.items() if reward == max_reward_movie]
        if max_rewad_filter:
         random_max_reward_movie = np.random.choice(max_rewad_filter)
        #print(random_max_reward_movie)
        for i in range(explore,iterations):
         #max_reward = max(movie_with_rewards, key=movie_with_rewards.get)  # Select movie with highest reward 
         max_rating_movie = user_movies.loc[user_movies['reward'] == user_movies['reward'].max()]    
         print(max_rating_movie)
         # Update rewards and counts         
         max_rating_movie_id = max_rating_movie['movieId'].iloc[0]  
         rating = max_rating_movie['rating'].iloc[0]
         
        #rating = movie_ratings.iloc[0]['rating']
         reward = self.compute_reward(rating)
         user_movies.loc[user_movies['movieId'] == max_rating_movie_id, 'reward'] += reward 
         #print(f"====For rewards  {i}: States: {movie_with_rewards}")
         total_rewards.append(reward)
         print(f"Exploiting {i}:User ID :{self.user_id} {user_movies}")
         #print(f"Selected movie: {max_reward} with reward: {movie_with_rewards[max_reward]}")
        
        return np.cumsum(total_rewards)
    def epsilon_greedy_policy(self,iterations, epsilon):
     
     user_movies = df[df['userId'] == self.user_id]
     user_movie_ids = user_movies['movieId'].unique()
     rewards_lst = []
     for _ in range(iterations):
        if np.random.rand() < epsilon:  # Exploration
            movieId = np.random.choice(user_movie_ids)
        else:  ##exploitation
            movieId = user_movies.loc[user_movies['movieId'] == user_movie_ids[np.argmax(user_movies['reward'].values)]]['movieId'].iloc[0]                     
        rating = user_movies.loc[user_movies['movieId'] == movieId]['rating'].iloc[0]
        #rating = movie_ratings.iloc[0]['rating']
        reward = self.compute_reward(rating)   
            
        user_movies.loc[user_movies['movieId'] == movieId, 'reward'] += reward 
        rewards_lst.append(reward)
     print(rewards_lst)
     return np.cumsum(rewards_lst)


In [164]:
env = MovieEnvironment(df)

env.reset()
 
# Recommend movies for users and run the simulation with random policy
random_cum_rewd  = env.random_policy(iterations=1000)
print(random_cum_rewd)


Recommendation for the user :224
[  1   2   3   4   5   6   7   8   8   9  10  10  11  12  13  14  15  16
  17  18  19  20  21  22  23  24  25  25  26  27  28  29  30  31  32  33
  34  34  35  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49
  50  51  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  65
  66  67  68  69  70  71  72  72  73  74  75  76  77  78  79  79  80  81
  82  83  84  85  86  87  88  88  89  90  90  91  92  93  94  95  96  96
  97  98  98  99 100 101 102 103 104 105 106 107 108 109 110 111 111 112
 113 114 115 116 117 118 119 120 121 122 123 124 125 126 126 127 128 129
 130 131 131 132 133 134 134 135 136 136 137 138 139 140 141 142 143 144
 145 146 146 147 148 149 149 149 149 150 150 151 152 152 153 154 155 156
 157 158 159 159 160 160 161 161 161 162 163 163 164 165 166 167 167 168
 169 170 171 172 173 173 174 175 176 177 178 179 180 181 182 183 184 185
 186 186 187 188 189 190 191 192 192 193 194 195 196 197 198 199 199 200
 201 202 203 203 2

In [165]:
# Recommend movies for all users and run the simulation with random policy
greedy_cum_rewd  = env.greedy_policy(iterations=1000)
print(greedy_cum_rewd)


       userId  movieId  rating  timestamp  reward
32915     224       39     5.0  971812919       2
32917     224      289     5.0  971812895       2
32918     224      356     5.0  971812827       2
32919     224      377     4.0  971812827       2
32920     224      440     4.0  971812860       2
32922     224      527     5.0  971812301       2
32923     224      593     4.0  971812319       2
32924     224      858     4.0  971812264       2
32925     224      953     4.0  971811286       2
32927     224     1197     5.0  971812860       2
32929     224     1259     5.0  971811884       2
32930     224     1265     5.0  971812765       2
32931     224     1270     5.0  971811286       2
32933     224     1307     5.0  971811933       2
32934     224     1393     5.0  971812919       2
32935     224     1394     5.0  971811933       2
32937     224     1721     5.0  971811594       2
32938     224     1968     5.0  971812118       2
32939     224     2028     5.0  971812185       2


In [166]:
eps1_greedy_policy_cum_rewards = env.epsilon_greedy_policy(iterations=1000,epsilon=0.1)
eps2_greedy_policy_cum_rewards = env.epsilon_greedy_policy(iterations=1000, epsilon=0.2)
eps5_greedy_policy_cum_rewards = env.epsilon_greedy_policy(iterations=1000, epsilon=0.5)
print(eps1_greedy_policy_cum_rewards)
print(eps2_greedy_policy_cum_rewards)
print(eps5_greedy_policy_cum_rewards)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [161]:
trace_random = go.Scatter(
    x=list(range(1000)),
    y=random_cum_rewd,
    mode='lines',
    name='Random Policy'
)

trace_greedy = go.Scatter(
    x=list(range(1000)),
    y=greedy_cum_rewd,
    mode='lines',
    name='Greedy Policy'
)
trace_eps_greedy_1 = go.Scatter(
    x=list(range(1000)),
    y=eps1_greedy_policy_cum_rewards,
    mode='lines',
    name='Epsilon Greedy Policy with eps 0.1'
)
trace_eps_greedy_2 = go.Scatter(
    x=list(range(1000)),
    y=eps2_greedy_policy_cum_rewards,
    mode='lines',
    name='Epsilon Greedy Policy with eps 0.2'
)

trace_eps_greedy_5 = go.Scatter(
    x=list(range(1000)),
    y=eps5_greedy_policy_cum_rewards,
    mode='lines',
    name='Epsilon Greedy Policy with eps 0.5'
)
fig = go.Figure(data=[trace_random,trace_greedy,trace_eps_greedy_1,trace_eps_greedy_2, trace_eps_greedy_5])

fig.update_layout(
    title="Cumulative Rewards Across Different Policies",
    xaxis_title="Number of Iterations",
    yaxis_title="Cumulative Reward",
    legend_title="Recommendation Policies",
    template="plotly_white",
    font=dict(size=14)
)

fig.show()